# atoMEC notebook for HZDR summer student lectures

atoMEC is a Python-based average-atom code for simulations of high energy density phenomena such as in warm dense matter.

In the following notebook, we will run through a few simple examples to demonstrate how atoMEC works and what information it gives us.

Thanks to Ekaterina Tsvetoslavova Stankulova for preparing the notebook.

### Installation and imports
We start by installing the atoMEC package from the GitHub repository.

In [ ]:
!pip3 install -q git+https://github.com/atomec-project/atoMEC.git@develop#egg=atoMEC

Next, we import some useful packages and the atoMEC modules that we will use.

In [ ]:
from math import *
import numpy as np
import mendeleev
import matplotlib.pyplot as plt
from pylab import *
from atoMEC import Atom, models, config

### Introduction and first example

The main object in atoMEC is the `Atom`, which takes as inputs important physical properties of the system. Of particular importance are the:

* Atomic species
* Temperature
* Mass density

In this example, you have to choose between Beryllium (by typing "Be" in the input box) and Iron (by typing "Fe").

Then the density (in units $\mathrm{g\ cm}^{-3}$): for reference, the room temperature densities of Beryllium and Iron are $1.85\mathrm{g\ cm}^{-3}$ and $7.87\mathrm{g\ cm}^{-3}$ respectively.

Finally, you must choose a temperature: we suggest using a temperature values between $1\times10^{-4}$ K and $1\times10^{-6}$ K, since this corresponds approximately to the warm dense matter temperature range.

The other important object in atoMEC is the choice of model. In this case, we select the `ISModel`. There are various choices of approximation within this model, but in this case we only consider the choice of boundary condition used to solve the Kohn-Sham (KS) equations.

So, the final input can be either "dirichlet" or "neumann" for the boundary condition.

In [ ]:
should_restart1 = True
should_restart2 = True
while should_restart1:
    should_restart1 = False
    # select the atom species
    atom_species = input("Which atom do you want to simulate?: ")
    # restrict choice to Be or Fe
    if atom_species in ["Be", "Fe"]:
        # input the density
        density = float(input("Density: "))
        # input the temperature
        temperature = float(input("Temperature: "))
        while should_restart2:
            should_restart2 = False
            # input the boundary condition
            boundary_condition = input("Which boundary condition do you want to use?: ").lower()
            # make sure boundary condition is valid
            if boundary_condition not in ["neumann", "dirichlet"]:
                print("Not a valid input")
                should_restart2 = True
            else:
                # initialize the main Atom object
                atom = Atom(atom_species, density=density, temp=temperature, units_temp="k")
                # choose the ISModel
                model = models.ISModel(atom, bc=boundary_condition)
                # run the self-consistent KS calculation
                output = model.CalcEnergy(35, 5, grid_params={"ngrid": 1000})
                # extract desired information
                print("Total free energy = ", output["energy"].F_tot)
                F = output["energy"].F_tot
                orbs = output["orbitals"].eigvals
    else:
        print("Not a valid input")
        should_restart1 = True

### Let's go further...

Now that you've seen a little how the code works and what is its output we'll do some plotting. In this example, we'll vary the density and observe the impact on the bound KS orbital eigenvalues. 

To do that you'll have to choose a set of densitites, for example 10 to 100 in steps of 10.

In [ ]:
should_restart1 = True
should_restart2 = True
while should_restart1:
    should_restart1 = False
    atom_species = input("Which atom do you want to simulate?: ")
    if atom_species in ["Be", "Fe"]:
        # start value of density
        density1 = float(input("Density start: "))
        # end value of density
        density2 = float(input("Density stop: "))
        # step size for cycling through density
        step = int(input("Step size: "))
        temperature = float(input("Temperature: "))
        while should_restart2:
            should_restart2 = False
            boundary_condition = input("Which boundary condition do you want to use?: ").lower()
            if boundary_condition not in ["neumann", "dirichlet"]:
                print("Not a valid input")
                should_restart2 = True
            else:
                print("Welcome to atoMEC! Please wait patiently until the calculation is done (it may take a while).")
                # initialize the eigenvalue array
                eigval = np.zeros((4, int(((density2 - density1)/step) + 1)))
                n = 0
                # iterate through the density range selected
                for i in arange(density1, density2 + 1, step):
                    # setting write_info=False stops output being printed
                    atom = Atom(atom_species, density=i, temp=temperature, units_temp="k", write_info=False)
                    model = models.ISModel(atom, bc=boundary_condition, write_info=False)
                    output = model.CalcEnergy(35, 5, grid_params={"ngrid": 1000}, write_info=False)
                    F = output["energy"].F_tot
                    eigs = output["orbitals"].eigvals
                    # select a few low-lying eigenvalues to plot
                    eigval[0, n] = eigs[0, 0, 0]
                    eigval[1, n] = eigs[0, 0, 1]
                    eigval[2, n] = eigs[0, 1, 0]
                    eigval[3, n] = eigs[0, 1, 1]
                    n = n + 1
                print("Done, now you can proceed with the plotting.")
    else:
        print("Not a valid input")
        should_restart1 = True

Now, let's plot the eigenvalues with respect to the density.

In [ ]:
den = arange(density1, density2 + 1, step)
plt.figure(1)
plt.plot(den, eigval[0, :], label = "$(l = 0, n = 1)$")
plt.plot(den, eigval[1, :], label = "$(l = 0, n = 2)$")
plt.plot(den, eigval[2, :], label = "$(l = 1, n = 2)$")
plt.plot(den, eigval[3, :], label = "$(l = 1, n = 3)$")
plt.title("Bound orbital eigenvalues vs density, for " + str(atom_species))
plt.ylabel("Orbital eigenvalues [Ha]"), plt.xlabel("Density $\mathrm{g\ cm}^{-3}$")
plt.legend()
plt.show()

fig, axs = plt.subplots(2, 2)
axs[0, 0].plot(den, eigval[0, :])
axs[0, 0].set_title("$(l = 0, n = 1)$")
axs[0, 1].plot(den, eigval[1, :])
axs[0, 1].set_title("$(l = 0, n = 2)$")
axs[1, 0].plot(den, eigval[2, :])
axs[1, 0].set_title("$(l = 1, n = 2)$")
axs[1, 1].plot(den, eigval[3, :])
axs[1, 1].set_title("$(l = 1, n = 3)$")

fig.text(0.5, 0.04, "Density ($\mathrm{g\ cm}^{-3}$)", ha='center')
fig.text(0.04, 0.5, "Orbit###al eigenvalues (Ha)", va='center', rotation='vertical')

#for ax in axs.flat:
 #   ax.set(xlabel="Density [g cm^-3]", ylabel="Orbital eigenvalues [Ha]")

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

### One last plot...

Now we are going to plot the density distribution with respect to radius and compare the result with both of the boundary conditions. You can choose between Beryllium and Iron like before. You can also choose the density and temperature.

In [ ]:
should_restart1 = True
while should_restart1:
    should_restart1 = False
    atom_species = input("Which atom do you want to simulate?: ")
    if atom_species in ["Be", "Fe"]:
        density = float(input("Density: "))
        temperature = float(input("Temperature: "))
        print("Please wait for the calculations to finish.")
        # usual set up of atom, model and energy calculation
        atom = Atom(atom_species, density=density, temp=temperature, units_temp="k", write_info=False)
        model = models.ISModel(atom, bc="dirichlet", write_info=False)
        output = model.CalcEnergy(35, 5, grid_params={"ngrid": 1000}, write_info=False)
        # extract and plot dirichlet density from file
        data = np.genfromtxt("density.csv", dtype=None, delimiter=' ', skip_header=1)
        plt.figure(1)
        plt.plot(data[:, 0], data[:, 0]**2*(data[:, 1] + data[:, 2]), label="Dirichlet boundary condition")
        model = models.ISModel(atom, bc="neumann", write_info=False)
        output = model.CalcEnergy(35, 5, grid_params={"ngrid": 1000}, write_info=False)
        # extract and plot neumann density from the output file
        data = np.genfromtxt("density.csv", dtype=None, delimiter=' ', skip_header=1)
        plt.plot(data[:, 0], data[:, 0]**2*(data[:, 1] + data[:, 2]), "--", label="Neumann boundary condition")
        plt.title("Density distribution")
        plt.xlabel("$r\ (a_0)$"), plt.ylabel("$r^2 n(r)\ (a_0^{-1})$")
        plt.xlim(0,atom.radius)
        plt.legend()
        plt.show()
    else:
        print("Not a valid input")
        should_restart1 = True

### If you want to do more...
You can try running atoMEC for a wider range of materials, densities and temperatures. Besides the eigenvalues (which you can think of as representing ionization energies), other interesting properties can be extracted from atoMEC. For example, there is a relationship between pressure and free energy (google is your friend). Can you think of a way to extract the pressure from the atoMEC output?

Useful links:
* atoMEC GitHub repository (source code): https://github.com/atomec-project/atoMEC
* atoMEC documentation: https://atomec-project.github.io/atoMEC/
* atoMEC notebooks: https://github.com/atomec-project/notebooks
* Paper on average-atom models: https://arxiv.org/abs/2103.09928